In [1]:
import pandas as pd
import numpy as np 
import plotly.express as px
import os

In [2]:
current_directory = os.getcwd()
customer_data = pd.ExcelFile(f'{current_directory}\\Customer_Churn_Data_Large.xlsx')
customer_data

In [3]:
customer_demographics  = pd.read_excel(customer_data, 'Customer_Demographics')
customer_demographics

,CustomerID,Age,Gender,MaritalStatus,IncomeLevel
0,1,62,M,Single,Low
1,2,65,M,Married,Low
2,3,18,M,Single,Low
3,4,21,M,Widowed,Low
4,5,21,M,Divorced,Medium
...,...,...,...,...,...
995,996,54,F,Single,Low
996,997,19,M,Widowed,High
997,998,47,M,Married,Low
998,999,23,M,Widowed,High


In [4]:
customer_demographics[['Gender', 'IncomeLevel']].value_counts().sort_index(ascending=False)

Gender  IncomeLevel
M       Medium         159
        Low            153
        High           175
F       Medium         167
        Low            172
        High           174
Name: count, dtype: int64

In [5]:
customer_demographics[['Age', 'IncomeLevel']].value_counts().sort_index(ascending=False)

Age  IncomeLevel
69   Medium          6
     Low             7
     High           10
68   Medium         11
     Low             8
                    ..
19   Low             7
     High            7
18   Medium         10
     Low            10
     High            4
Name: count, Length: 156, dtype: int64

In [6]:
counts = (
    customer_demographics[['Age', 'IncomeLevel']]
    .value_counts()
    .sort_index(ascending=False)
)

df = pd.DataFrame(counts).reset_index()
df



,Age,IncomeLevel,count
0,69,Medium,6
1,69,Low,7
2,69,High,10
3,68,Medium,11
4,68,Low,8
...,...,...,...
151,19,Low,7
152,19,High,7
153,18,Medium,10
154,18,Low,10


In [7]:
fig = px.treemap(   
    df, 
    path=['IncomeLevel', 'Age'],
    title='Customer Income and Age Demographics',
    values='count',
    labels={'IncomeLevel': 'Income Level', 'Age': 'Age', 'count': 'Count'},
    color='count',
    color_continuous_scale='Blues',
)


df['custom_text'] = 'Age: ' + df['Age'].astype(str) + ' | Count: ' + df['count'].astype(str)

fig = px.treemap(
    df,
    path=['IncomeLevel', 'Age'],
    values='count',
    color='count',
    color_continuous_scale='Blues',
    custom_data=['Age', 'count'],

)

# Set custom text on tiles using 'text'
fig.update_traces(
    text=df['custom_text'],  # Shows directly on the treemap block
    textinfo='text',         # Use 'text' instead of label/value
    textfont_size=8,
    hovertemplate='<b>Income Level:</b> %{parent}<br>' +
                    '<b>Age:</b> %{customdata[0]}<br>' +
                    '<b>Count:</b> %{customdata[1]}<extra></extra>'
)

fig.update_layout(width=1200, height=1000,title='Customer Income and Age Demographics', margin=dict(t=50, l=25, r=25, b=25))
fig.show()

In [8]:
customer_demographics['IncomeLevel'].value_counts().sort_values(ascending=False).head(20)

counts = (
    (customer_demographics['IncomeLevel']
    .value_counts()
    .sort_values(ascending=False)/len(customer_demographics))*100
)

# Convert to DataFrame for Plotly
df = counts.reset_index()
df.columns = ['IncomeLevel', 'count']

# Create the bar chart
fig = px.bar(
    df,
    x='IncomeLevel',
    y='count',
    title='Income Level Distribution',
    text='count',
)

fig.update_layout(
    xaxis_title='Income Level',
    yaxis_title='Number of Customers',
    xaxis_tickangle=45
)

fig.update_traces(marker_color='teal', textposition='outside')
fig.update_layout(width=700)

fig.show()



In [9]:
transaction_history  = pd.read_excel(customer_data, 'Transaction_History')
transaction_history

,CustomerID,TransactionID,TransactionDate,AmountSpent,ProductCategory
0,1,7194,2022-03-27,416.50,Electronics
1,2,7250,2022-08-08,54.96,Clothing
2,2,9660,2022-07-25,197.50,Electronics
3,2,2998,2022-01-25,101.31,Furniture
4,2,1228,2022-07-24,397.37,Clothing
...,...,...,...,...,...
5049,1000,2724,2022-09-08,232.06,Groceries
5050,1000,2917,2022-12-13,324.98,Books
5051,1000,2979,2022-06-15,375.34,Groceries
5052,1000,8594,2022-04-08,166.73,Books


In [10]:
transaction_history['CustomerID'].unique().shape[0]

1000

In [11]:
transaction_history['ProductCategory'].value_counts().sort_values(ascending=False)   

ProductCategory
Books          1041
Groceries      1020
Electronics    1001
Clothing       1000
Furniture       992
Name: count, dtype: int64

In [12]:
import pandas as pd
import plotly.express as px

# Convert TransactionDate to datetime (if not already)
transaction_history['TransactionDate'] = pd.to_datetime(transaction_history['TransactionDate'])

# Optional: Group by week or month to make the chart clearer
transaction_history['Period'] = transaction_history['TransactionDate'].dt.to_period('M').astype(str)

# Aggregate number of transactions per category per period
category_sales = (
    transaction_history
    .groupby(['Period', 'ProductCategory'])
    .size()
    .reset_index(name='TransactionCount')
)

# Line chart using Plotly
fig = px.line(
    category_sales,
    x='Period',
    y='TransactionCount',
    color='ProductCategory',
    title='Top-Selling Product Categories Over Time',
    markers=True
)

# Improve layout
fig.update_layout(
    xaxis_title='Period (Monthly)',
    yaxis_title='Number of Transactions',
    hovermode='x unified',
    width=1200,
    height=400
)
fig.update_traces(mode='lines+markers', marker=dict(size=5))
fig.update_layout(legend_title_text='Product Category')
fig.update_xaxes(tickangle=45, dtick='M1', tickformat='%b')
fig.show()


In [13]:
most_spent_products = transaction_history.groupby('ProductCategory')['AmountSpent'].sum().reset_index().sort_values(by='AmountSpent', ascending=False)
most_spent_products['Percentage'] = most_spent_products['AmountSpent']/transaction_history['AmountSpent'].sum() * 100
most_spent_products


fig = px.pie(
    most_spent_products, 
    names='ProductCategory',
    values='Percentage',
    
)

fig.show()

In [14]:
transaction_history['TransactionDate'].dt.month.value_counts().sort_index()

# number_of_interactions = transaction_history.groupby('CustomerID').size()
number_of_transactions = transaction_history.groupby('CustomerID').size().reset_index(name='NumberOfInteractions')
number_of_transactions

,CustomerID,NumberOfInteractions
0,1,1
1,2,7
2,3,6
3,4,5
4,5,8
...,...,...
995,996,1
996,997,2
997,998,1
998,999,9


In [15]:
# Sum of AmountSpent per CustomerID
customers_total_spent = transaction_history.groupby('CustomerID')['AmountSpent'].sum().reset_index().sort_values(by='AmountSpent', ascending=False)

# Optional: rename for clarity
customers_total_spent.rename(columns={'AmountSpent': 'TotalSpent'}, inplace=True)
customers_total_spent

,CustomerID,TotalSpent
480,481,3386.04
368,369,3352.34
715,716,3220.30
36,37,3166.88
517,518,3078.60
...,...,...
337,338,13.86
147,148,13.04
262,263,13.04
646,647,12.46


In [16]:
transaction_history['AmountSpent'].describe()


count    5054.000000
mean      250.707351
std       142.250838
min         5.180000
25%       127.105000
50%       250.525000
75%       373.412500
max       499.860000
Name: AmountSpent, dtype: float64

In [17]:
customer_service = pd.read_excel(customer_data, 'Customer_Service')
customer_service

,CustomerID,InteractionID,InteractionDate,InteractionType,ResolutionStatus
0,1,6363,2022-03-31,Inquiry,Resolved
1,2,3329,2022-03-17,Inquiry,Resolved
2,3,9976,2022-08-24,Inquiry,Resolved
3,4,7354,2022-11-18,Inquiry,Resolved
4,4,5393,2022-07-03,Inquiry,Unresolved
...,...,...,...,...,...
997,990,3671,2022-10-25,Complaint,Unresolved
998,992,2114,2022-09-29,Feedback,Unresolved
999,994,3087,2022-07-02,Complaint,Unresolved
1000,994,8508,2022-05-14,Complaint,Unresolved


In [18]:
customer_service['InteractionDate'].dt.month.value_counts().sort_index()

number_of_interactions = customer_service.groupby('CustomerID').size().reset_index(name='NumberOfInteractions')
number_of_interactions_per_month = {}

In [19]:
customer_service['InteractionType'].value_counts().sort_values(ascending=False)

InteractionType
Feedback     360
Complaint    335
Inquiry      307
Name: count, dtype: int64

In [20]:
customer_service['InteractionMonth'] = customer_service['InteractionDate'].dt.month_name()

monthly_counts = (
    customer_service
    .groupby(['InteractionType', 'InteractionMonth'])
    .size()
    .sort_index()
    .unstack(fill_value=0)
)

monthly_line_df = pd.DataFrame(monthly_counts)

monthly_line_df.sort_index(inplace=True)
monthly_line_df = monthly_line_df.reindex(
    ['January', 'February', 'March', 'April', 'May', 'June', 
        'July', 'August', 'September', 'October', 'November', 'December'], 
    axis=1
)   
monthly_line_df.reset_index(inplace=True)
monthly_line_df

InteractionMonth,InteractionType,January,February,March,April,May,June,July,August,September,October,November,December
0,Complaint,25,22,31,25,41,32,33,26,28,32,27,13
1,Feedback,27,26,36,32,20,31,42,28,33,31,25,29
2,Inquiry,25,20,26,23,26,24,22,28,21,26,32,34


In [21]:

monthly_line_long = monthly_line_df.melt(id_vars='InteractionType', var_name='Month', value_name='Count')

monthly_line_long['Month'] = monthly_line_long['Month']

# Sort for proper line plot rendering
monthly_line_long = monthly_line_long.sort_index()

# Plot the line chart
fig = px.line(
    monthly_line_long,
    x='Month',
    y='Count',
    color='InteractionType',
    markers=True,
    title='Monthly Customer Service Interactions by Type',
    labels={'Month': 'Month', 'Count': 'Number of Interactions'}
)

fig.update_layout(xaxis=dict(tickmode='linear', dtick=1))
fig.show()

In [22]:
customer_service['ResolutionStatus'].value_counts().sort_values(ascending=False)

ResolutionStatus
Resolved      523
Unresolved    479
Name: count, dtype: int64

In [23]:
online_activity = pd.read_excel(customer_data, 'Online_Activity')
online_activity

,CustomerID,LastLoginDate,LoginFrequency,ServiceUsage
0,1,2023-10-21,34,Mobile App
1,2,2023-12-05,5,Website
2,3,2023-11-15,3,Website
3,4,2023-08-25,2,Website
4,5,2023-10-27,41,Website
...,...,...,...,...
995,996,2023-01-29,38,Mobile App
996,997,2023-04-01,5,Mobile App
997,998,2023-07-10,47,Website
998,999,2023-01-08,23,Website


In [24]:
online_activity.describe()

,CustomerID,LastLoginDate,LoginFrequency
count,1000.000000,1000,1000.000000
mean,500.500000,2023-07-05 21:28:48,25.912000
min,1.000000,2023-01-01 00:00:00,1.000000
25%,250.750000,2023-04-08 00:00:00,13.750000
50%,500.500000,2023-07-10 12:00:00,27.000000
75%,750.250000,2023-10-01 06:00:00,38.000000
max,1000.000000,2023-12-31 00:00:00,49.000000
std,288.819436,NaN,14.055953


In [25]:
login_df = online_activity['LoginFrequency'].groupby(online_activity['ServiceUsage']).sum().sort_values(ascending=False).reset_index()
login_df.rename(columns={'LoginFrequency': 'TotalLoginFrequency'}, inplace=True)
login_df

fig = px.bar(
    login_df,
    x='ServiceUsage',
    y='TotalLoginFrequency',
    title='Total Login Frequency by Service Usage',
    text='TotalLoginFrequency',
)

fig.update_layout(
    xaxis_title='Service Type',
    yaxis_title='Number of Users',
    xaxis_tickangle=45,
    width=500,
    height=500
)

fig.show()

In [26]:
service_usage = online_activity['ServiceUsage'].value_counts().sort_values(ascending=False).reset_index()

In [27]:
fig = px.bar(
    service_usage, 
    x='ServiceUsage', 
    y='count', 
    title='Service Usage Distribution',
    labels={'index': 'Service Type', 'ServiceUsage': 'Number of Users'},
    text='count'
)

fig.update_layout(
    xaxis_title='Service Type',
    yaxis_title='Number of Users',
    xaxis_tickangle=45,
    width=600,
    height=600
)
fig.show()

In [28]:
# Ensure LastLoginDate is datetime
online_activity['LastLoginDate'] = pd.to_datetime(online_activity['LastLoginDate'])

# Create a Year-Month column for grouping
online_activity['YearMonth'] = online_activity['LastLoginDate'].dt.to_period('M').astype(str)

# Count logins per month
monthly_counts = online_activity['YearMonth'].value_counts().sort_index()
monthly_counts_df = monthly_counts.reset_index()
monthly_counts_df.columns = ['YearMonth', 'LoginCount']

# Plot the line chart
fig = px.line(
    monthly_counts_df, 
    x='YearMonth', 
    y='LoginCount',
    title='Monthly Last Login Counts',
    labels={'YearMonth': 'Month', 'LoginCount': 'Number of Logins'}
)
fig.update_traces(mode='lines+markers')
fig.update_xaxes(type='category')  # Ensures months are shown in order

fig.show()

In [29]:
churn_status = pd.read_excel(customer_data, 'Churn_Status')
churn_status


,CustomerID,ChurnStatus
0,1,0
1,2,1
2,3,0
3,4,0
4,5,0
...,...,...
995,996,0
996,997,0
997,998,0
998,999,0


In [30]:
churn_status_copy = churn_status.copy()
churn_status_copy['ChurnStatus'] = churn_status_copy['ChurnStatus'].astype('bool')
churn_status_copy['ChurnStatus'] = churn_status_copy['ChurnStatus'].replace({True: 'Churned', False: 'Active'})
churn_status_copy

,CustomerID,ChurnStatus
0,1,Active
1,2,Churned
2,3,Active
3,4,Active
4,5,Active
...,...,...
995,996,Active
996,997,Active
997,998,Active
998,999,Active


In [31]:

churn_status_copy['ChurnStatus'].value_counts().sort_values(ascending=False)

ChurnStatus
Active     796
Churned    204
Name: count, dtype: int64

In [32]:
churn_status_copy[churn_status_copy['ChurnStatus'] == 'Churned'].count()/churn_status_copy['ChurnStatus'].count()*100

CustomerID     20.4
ChurnStatus    20.4
dtype: float64

In [33]:
churn_status_copy[churn_status_copy['ChurnStatus'] == 'Active'].count()/churn_status_copy['ChurnStatus'].count()*100

CustomerID     79.6
ChurnStatus    79.6
dtype: float64

In [34]:
churn_counts = churn_status_copy['ChurnStatus'].value_counts().sort_values(ascending=False)
fig = px.pie(
    names=churn_counts.index,
    values=churn_counts.values,
    title="Churn Status Distribution"
)
fig.show()

In [35]:
first_customer_transaction = transaction_history.groupby('CustomerID')['TransactionDate'].min().reset_index()
first_customer_transaction.rename(columns={'TransactionDate': 'FirstTransactionDate'}, inplace=True)
first_customer_transaction

,CustomerID,FirstTransactionDate
0,1,2022-03-27
1,2,2022-01-09
2,3,2022-02-11
3,4,2022-05-22
4,5,2022-02-21
...,...,...
995,996,2022-07-24
996,997,2022-08-06
997,998,2022-09-18
998,999,2022-01-20


In [36]:
last_customer_transaction = transaction_history.groupby('CustomerID')['TransactionDate'].max().reset_index()
last_customer_transaction.rename(columns={'TransactionDate': 'LastTransactionDate'}, inplace=True)
last_customer_transaction

,CustomerID,LastTransactionDate
0,1,2022-03-27
1,2,2022-11-19
2,3,2022-10-08
3,4,2022-12-27
4,5,2022-12-21
...,...,...
995,996,2022-07-24
996,997,2022-10-25
997,998,2022-09-18
998,999,2022-12-07


In [37]:
# Count of online actions
online_activity_count = online_activity[['CustomerID', 'LoginFrequency']].drop_duplicates('CustomerID')
transaction_count_per_customer = (
    transaction_history
    .groupby('CustomerID')
    .size()
    .reset_index(name='TransactionCount')
)
# Most recent login date
last_login_df = (
    online_activity
    .groupby('CustomerID')['LastLoginDate']
    .max()
    .reset_index(name='LastLoginDate')
)



# Merge both
merged_df = (
    customer_demographics
    .merge(
        transaction_history.groupby('CustomerID')['AmountSpent'].sum().reset_index(name='TotalSpent'),
        on='CustomerID',
        how='left'
    )
    .merge(
        transaction_count_per_customer,
        on='CustomerID',
        how='left'
    )
    .merge(
        first_customer_transaction,
        on='CustomerID',
        how='left'
    )
    .merge(
        last_customer_transaction,
        on='CustomerID',
        how='left'
    )
    .merge(last_login_df, on='CustomerID', how='left')
    .merge(
        customer_service.groupby('CustomerID').size().reset_index(name='NumServiceInteractions'),
        on='CustomerID',
        how='left'
    )
    .merge(online_activity_count, on='CustomerID', how='left')
    .merge(online_activity[['CustomerID', 'ServiceUsage']].drop_duplicates('CustomerID'), on='CustomerID', how='left')
    .merge(churn_status, on='CustomerID', how='left')
)


In [38]:
merged_df.head()

,CustomerID,Age,Gender,MaritalStatus,IncomeLevel,TotalSpent,TransactionCount,FirstTransactionDate,LastTransactionDate,LastLoginDate,NumServiceInteractions,LoginFrequency,ServiceUsage,ChurnStatus
0,1,62,M,Single,Low,416.50,1,2022-03-27,2022-03-27,2023-10-21,1.0,34,Mobile App,0
1,2,65,M,Married,Low,1547.42,7,2022-01-09,2022-11-19,2023-12-05,1.0,5,Website,1
2,3,18,M,Single,Low,1702.98,6,2022-02-11,2022-10-08,2023-11-15,1.0,3,Website,0
3,4,21,M,Widowed,Low,917.29,5,2022-05-22,2022-12-27,2023-08-25,2.0,2,Website,0
4,5,21,M,Divorced,Medium,2001.49,8,2022-02-21,2022-12-21,2023-10-27,NaN,41,Website,0


In [39]:
merged_df.isnull().sum()


CustomerID                  0
Age                         0
Gender                      0
MaritalStatus               0
IncomeLevel                 0
TotalSpent                  0
TransactionCount            0
FirstTransactionDate        0
LastTransactionDate         0
LastLoginDate               0
NumServiceInteractions    332
LoginFrequency              0
ServiceUsage                0
ChurnStatus                 0
dtype: int64

In [40]:
merged_df_copy = merged_df.copy()

merged_df_copy.fillna({'LastLoginDate': merged_df_copy['LastLoginDate'].median(),
                        'ServiceUsage': 'Unknown',
                        'TotalSpent': merged_df_copy['TotalSpent'].mean(),
                        'NumServiceInteractions': 0}, inplace=True
                        )
merged_df_copy


,CustomerID,Age,Gender,MaritalStatus,IncomeLevel,TotalSpent,TransactionCount,FirstTransactionDate,LastTransactionDate,LastLoginDate,NumServiceInteractions,LoginFrequency,ServiceUsage,ChurnStatus
0,1,62,M,Single,Low,416.50,1,2022-03-27,2022-03-27,2023-10-21,1.0,34,Mobile App,0
1,2,65,M,Married,Low,1547.42,7,2022-01-09,2022-11-19,2023-12-05,1.0,5,Website,1
2,3,18,M,Single,Low,1702.98,6,2022-02-11,2022-10-08,2023-11-15,1.0,3,Website,0
3,4,21,M,Widowed,Low,917.29,5,2022-05-22,2022-12-27,2023-08-25,2.0,2,Website,0
4,5,21,M,Divorced,Medium,2001.49,8,2022-02-21,2022-12-21,2023-10-27,0.0,41,Website,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,54,F,Single,Low,227.25,1,2022-07-24,2022-07-24,2023-01-29,0.0,38,Mobile App,0
996,997,19,M,Widowed,High,419.82,2,2022-08-06,2022-10-25,2023-04-01,0.0,5,Mobile App,0
997,998,47,M,Married,Low,252.15,1,2022-09-18,2022-09-18,2023-07-10,0.0,47,Website,0
998,999,23,M,Widowed,High,2393.26,9,2022-01-20,2022-12-07,2023-01-08,0.0,23,Website,0


In [41]:
merged_df_copy.isnull().sum()


CustomerID                0
Age                       0
Gender                    0
MaritalStatus             0
IncomeLevel               0
TotalSpent                0
TransactionCount          0
FirstTransactionDate      0
LastTransactionDate       0
LastLoginDate             0
NumServiceInteractions    0
LoginFrequency            0
ServiceUsage              0
ChurnStatus               0
dtype: int64

In [42]:
px.box(merged_df, x='ChurnStatus', y='TotalSpent')


In [43]:
# Churn rate by income level
churn_by_income = merged_df_copy.groupby('IncomeLevel')['ChurnStatus'].mean().sort_values(ascending=False).reset_index()
fig = px.bar(churn_by_income, x='IncomeLevel', y='ChurnStatus', 
        title='Churn Rate Probability by Income Level', labels={'ChurnStatus':'Churn Probability'})

fig.update_layout(
        xaxis_title='Income Level',
        yaxis_title='Churn Probability',
        xaxis_tickangle=45,
        width=500,
        height=500
)

fig.show()

In [44]:
bins = [merged_df['Age'].min(), 25, 35, 45, 55, 65, merged_df['Age'].max()]
labels = [f'{merged_df["Age"].min()}-24', '25-34', '35-44', '45-54', '55-64', f'65-{merged_df["Age"].max()}']

#new column for age groups
merged_df_copy['AgeGroup'] = pd.cut(merged_df['Age'], bins=bins, right=False, labels=labels)

# Group by AgeGroup and get the mean churn rate
churn_by_age_group = merged_df_copy.groupby('AgeGroup')['ChurnStatus'].mean().reset_index()

# Plot using bar chart
fig = px.bar(
    churn_by_age_group,
    x='AgeGroup',
    y='ChurnStatus',
    title='Mean Churn Rate by Age Group',
    labels={'ChurnStatus': 'Mean Churn Probability', 'AgeGroup': 'Age Group'}
)

fig.update_layout(
        xaxis_title='Age',
        yaxis_title='Churn Probability',
        xaxis_tickangle=45,
        width=800,
        height=500
)

fig.show()


C:\Users\Admin\AppData\Local\Temp\ipykernel_13280\447999251.py:8: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [45]:
merged_df

,CustomerID,Age,Gender,MaritalStatus,IncomeLevel,TotalSpent,TransactionCount,FirstTransactionDate,LastTransactionDate,LastLoginDate,NumServiceInteractions,LoginFrequency,ServiceUsage,ChurnStatus
0,1,62,M,Single,Low,416.50,1,2022-03-27,2022-03-27,2023-10-21,1.0,34,Mobile App,0
1,2,65,M,Married,Low,1547.42,7,2022-01-09,2022-11-19,2023-12-05,1.0,5,Website,1
2,3,18,M,Single,Low,1702.98,6,2022-02-11,2022-10-08,2023-11-15,1.0,3,Website,0
3,4,21,M,Widowed,Low,917.29,5,2022-05-22,2022-12-27,2023-08-25,2.0,2,Website,0
4,5,21,M,Divorced,Medium,2001.49,8,2022-02-21,2022-12-21,2023-10-27,NaN,41,Website,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,54,F,Single,Low,227.25,1,2022-07-24,2022-07-24,2023-01-29,NaN,38,Mobile App,0
996,997,19,M,Widowed,High,419.82,2,2022-08-06,2022-10-25,2023-04-01,NaN,5,Mobile App,0
997,998,47,M,Married,Low,252.15,1,2022-09-18,2022-09-18,2023-07-10,NaN,47,Website,0
998,999,23,M,Widowed,High,2393.26,9,2022-01-20,2022-12-07,2023-01-08,NaN,23,Website,0


In [46]:
merged_df_copy['ChurnStatus'] = merged_df_copy['ChurnStatus'].astype('bool')
merged_df_copy['ChurnStatus'] = merged_df_copy['ChurnStatus'].replace({True: 'True', False: 'False'})

In [47]:
# One-hot encoding for ML readiness
merged_df_copy = pd.get_dummies(merged_df, columns=['Gender', 'MaritalStatus', 'IncomeLevel'])
merged_df_copy


,CustomerID,Age,TotalSpent,TransactionCount,FirstTransactionDate,LastTransactionDate,LastLoginDate,NumServiceInteractions,LoginFrequency,ServiceUsage,ChurnStatus,Gender_F,Gender_M,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,MaritalStatus_Widowed,IncomeLevel_High,IncomeLevel_Low,IncomeLevel_Medium
0,1,62,416.50,1,2022-03-27,2022-03-27,2023-10-21,1.0,34,Mobile App,0,False,True,False,False,True,False,False,True,False
1,2,65,1547.42,7,2022-01-09,2022-11-19,2023-12-05,1.0,5,Website,1,False,True,False,True,False,False,False,True,False
2,3,18,1702.98,6,2022-02-11,2022-10-08,2023-11-15,1.0,3,Website,0,False,True,False,False,True,False,False,True,False
3,4,21,917.29,5,2022-05-22,2022-12-27,2023-08-25,2.0,2,Website,0,False,True,False,False,False,True,False,True,False
4,5,21,2001.49,8,2022-02-21,2022-12-21,2023-10-27,NaN,41,Website,0,False,True,True,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,54,227.25,1,2022-07-24,2022-07-24,2023-01-29,NaN,38,Mobile App,0,True,False,False,False,True,False,False,True,False
996,997,19,419.82,2,2022-08-06,2022-10-25,2023-04-01,NaN,5,Mobile App,0,False,True,False,False,False,True,True,False,False
997,998,47,252.15,1,2022-09-18,2022-09-18,2023-07-10,NaN,47,Website,0,False,True,False,True,False,False,False,True,False
998,999,23,2393.26,9,2022-01-20,2022-12-07,2023-01-08,NaN,23,Website,0,False,True,False,False,False,True,True,False,False


In [48]:
print(f"""Churn Risk Hotspots:
- High Income Churn Rate: {churn_by_income[churn_by_income['IncomeLevel']=='High']['ChurnStatus'].values[0]:.1%}
- Low Login Churners: {merged_df[(merged_df['LoginFrequency']<3) & merged_df['ChurnStatus']].shape[0]} customers""")


Churn Risk Hotspots:
- High Income Churn Rate: 19.2%
- Low Login Churners: 13 customers


In [49]:
from datetime import datetime
merged_df_copy['Tenure'] = (
    pd.to_datetime(merged_df['LastLoginDate']) -  # Project's reference date
    pd.to_datetime(merged_df['FirstTransactionDate'])
).dt.days

In [50]:
merged_df_copy

,CustomerID,Age,TotalSpent,TransactionCount,FirstTransactionDate,LastTransactionDate,LastLoginDate,NumServiceInteractions,LoginFrequency,ServiceUsage,...,Gender_F,Gender_M,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,MaritalStatus_Widowed,IncomeLevel_High,IncomeLevel_Low,IncomeLevel_Medium,Tenure
0,1,62,416.50,1,2022-03-27,2022-03-27,2023-10-21,1.0,34,Mobile App,...,False,True,False,False,True,False,False,True,False,573
1,2,65,1547.42,7,2022-01-09,2022-11-19,2023-12-05,1.0,5,Website,...,False,True,False,True,False,False,False,True,False,695
2,3,18,1702.98,6,2022-02-11,2022-10-08,2023-11-15,1.0,3,Website,...,False,True,False,False,True,False,False,True,False,642
3,4,21,917.29,5,2022-05-22,2022-12-27,2023-08-25,2.0,2,Website,...,False,True,False,False,False,True,False,True,False,460
4,5,21,2001.49,8,2022-02-21,2022-12-21,2023-10-27,NaN,41,Website,...,False,True,True,False,False,False,False,False,True,613
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,54,227.25,1,2022-07-24,2022-07-24,2023-01-29,NaN,38,Mobile App,...,True,False,False,False,True,False,False,True,False,189
996,997,19,419.82,2,2022-08-06,2022-10-25,2023-04-01,NaN,5,Mobile App,...,False,True,False,False,False,True,True,False,False,238
997,998,47,252.15,1,2022-09-18,2022-09-18,2023-07-10,NaN,47,Website,...,False,True,False,True,False,False,False,True,False,295
998,999,23,2393.26,9,2022-01-20,2022-12-07,2023-01-08,NaN,23,Website,...,False,True,False,False,False,True,True,False,False,353


In [51]:
# Churn rate by tenure quartile
merged_df_copy['TenureQuartile'] = pd.qcut(merged_df_copy['Tenure'], 4, labels=['Q1','Q2','Q3','Q4'])
churn_by_tenure = merged_df_copy.groupby('TenureQuartile')['ChurnStatus'].mean()
fig = px.bar(
    churn_by_tenure.reset_index(),
    x='TenureQuartile',
    y='ChurnStatus',
    title='Churn Rate by Tenure Quartile',
    labels={'ChurnStatus': 'Churn Probability', 'TenureQuartile': 'Tenure Quartile'}
)

fig.show()

C:\Users\Admin\AppData\Local\Temp\ipykernel_13280\1226170124.py:3: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [52]:
merged_df_copy = merged_df.copy()

merged_df_copy.fillna({'LastLoginDate': merged_df_copy['LastLoginDate'].median(),
                        'ServiceUsage': 'Unknown',
                        'TotalSpent': merged_df_copy['TotalSpent'].mean(),
                        'NumServiceInteractions': 0}, inplace=True
                        )

merged_df_copy.isnull().sum()

CustomerID                0
Age                       0
Gender                    0
MaritalStatus             0
IncomeLevel               0
TotalSpent                0
TransactionCount          0
FirstTransactionDate      0
LastTransactionDate       0
LastLoginDate             0
NumServiceInteractions    0
LoginFrequency            0
ServiceUsage              0
ChurnStatus               0
dtype: int64

In [53]:
merged_df_copy.to_csv('cleaned_customer_data.csv', index=False)